In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime
from numpy import array


     |████████████████████████████████| 6.4 MB 24.5 MB/s 
     |████████████████████████████████| 63 kB 590 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 3.3 MB/s 


In [22]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
    'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in','all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(11,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-12][0],
                df[each-12][3],
                df[each-12][4],
                df[each-11][0],
                df[each-11][3],
                df[each-11][4],
                df[each-10][0],
                df[each-10][3],
                df[each-10][4],
                df[each-9][0],
                df[each-9][3],
                df[each-9][4],
                df[each-8][0],
                df[each-8][3],
                df[each-8][4],
                df[each-7][0],
                df[each-7][3],
                df[each-7][4],
                df[each-6][0],
                df[each-6][3],
                df[each-6][4],
                df[each-5][0],
                df[each-5][3],
                df[each-5][4],
                df[each-4][0],
                df[each-4][3],
                df[each-4][4],
                df[each-3][0],
                df[each-3][3],
                df[each-3][4],
                df[each-2][0],
                df[each-2][3],
                df[each-2][4],
                df[each-1][0],
                df[each-1][3],
                df[each-1][4],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(1,10))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["suggestion"] = last_column
  return scaled
def column_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  df_scaled = scaler.fit_transform(df.to_numpy())
  df_scaled = pd.DataFrame(df_scaled)
  df_scaled["suggestion"] = last_column
  return df_scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      #df = column_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df   

In [ ]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

symbols = get_crypto_syms()
#symbols = read_syms_from_txt()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 13:
          dfs.append(df)

data = process(dfs)
xTrain, xTest, yTrain, yTest = spliting(data)
data

In [27]:
data

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,suggestion
0,10.000000,9.962342,9.962342,1.011749,1.011339,1.011339,1.011252,1.005053,1.005053,1.004991,...,1.005890,1.004207,1.004207,1.004180,1.003210,1.003210,1.003046,1.002284,1.002284,0.0
1,10.000000,9.725496,9.725496,9.666847,5.515231,5.515231,5.473647,2.146474,2.146474,2.131205,...,4.930512,4.281240,4.281240,4.171352,3.660986,3.660986,3.654794,1.000000,1.000000,0.0
2,10.000000,5.736592,5.736592,5.693888,2.277124,2.277124,2.261444,4.075505,4.075505,3.971105,...,4.356526,3.832417,3.832417,3.826059,1.099779,1.099779,1.066812,1.000000,1.000000,1.0
3,7.035992,2.642286,2.642286,2.622124,4.954872,4.954872,4.820620,3.236222,3.236222,3.281911,...,4.634102,1.128308,1.128308,1.085915,1.000000,1.000000,1.104318,2.741943,2.741943,0.0
4,2.622124,4.954872,4.954872,4.820620,3.236222,3.236222,3.281911,4.226854,4.226854,4.216962,...,1.085915,1.000000,1.000000,1.104318,2.741943,2.741943,2.822847,2.140218,2.140218,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765264,1.642061,3.063500,3.063500,3.061348,1.052867,1.052867,1.054477,1.000000,1.000000,1.002004,...,3.074995,8.558728,8.558728,8.558176,10.000000,10.000000,9.999884,5.583475,5.583475,0.0
1765265,3.937438,2.182552,2.182552,2.183959,2.136360,2.136360,2.138111,2.463915,2.463915,2.463166,...,8.740223,10.000000,10.000000,9.999899,6.141116,6.141116,6.140918,1.000000,1.000000,1.0
1765266,2.183959,2.136360,2.136360,2.138111,2.463915,2.463915,2.463166,6.006860,6.006860,6.011186,...,9.999899,6.141116,6.141116,6.140918,1.000000,1.000000,1.000924,3.114194,3.114194,1.0
1765267,2.138111,2.463915,2.463915,2.463166,6.006860,6.006860,6.011186,2.630554,2.630554,2.630223,...,6.140918,1.000000,1.000000,1.000924,3.114194,3.114194,3.114139,5.564232,5.564232,0.0


In [28]:
model = Sequential()


model.add(Dense(4096, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 4096)              151552    
                                                                 
 dense_28 (Dense)            (None, 1024)              4195328   
                                                                 
 dense_29 (Dense)            (None, 256)               262400    
                                                                 
 dense_30 (Dense)            (None, 100)               25700     
                                                                 
 dense_31 (Dense)            (None, 1)                 101       
                                                                 
Total params: 4,635,081
Trainable params: 4,635,081
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=200,batch_size=20000,validation_data=(xTest,yTest))


Epoch 1/200
194/194 [==============================] - 12s 59ms/step - loss: 0.7243 - accuracy: 0.5272 - val_loss: 0.6875 - val_accuracy: 0.5420
Epoch 2/200
194/194 [==============================] - 11s 58ms/step - loss: 0.6863 - accuracy: 0.5457 - val_loss: 0.6847 - val_accuracy: 0.5502
Epoch 3/200
194/194 [==============================] - 11s 59ms/step - loss: 0.6836 - accuracy: 0.5542 - val_loss: 0.6817 - val_accuracy: 0.5584
Epoch 4/200
194/194 [==============================] - 11s 58ms/step - loss: 0.6810 - accuracy: 0.5606 - val_loss: 0.6796 - val_accuracy: 0.5632
Epoch 5/200
194/194 [==============================] - 12s 61ms/step - loss: 0.6783 - accuracy: 0.5659 - val_loss: 0.6761 - val_accuracy: 0.5697
Epoch 6/200
194/194 [==============================] - 12s 61ms/step - loss: 0.6744 - accuracy: 0.5739 - val_loss: 0.6750 - val_accuracy: 0.5715
Epoch 7/200
194/194 [==============================] - 12s 60ms/step - loss: 0.6708 - accuracy: 0.5792 - val_loss: 0.6697 - val_ac

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")


Mounted at /content/gdrive/


In [ ]:
address = "/content/gdrive/MyDrive/Colab Files/12c_05sclaer.csv"
data = pd.read_csv(address)

data

,Unnamed: 0,12_candels_close,11_candels_close,10_candels_close,9_candels_close,8_candels_close,7_candels_close,6_candels_close,5_candels_close,4_candels_close,3_candels_close,2_candels_close,1_candels_close,suggestion
0,0,0.949209,2.169026,2.826810,5.000000,4.633638,4.375520,4.304747,1.794337,1.153204,0.000000,0.432971,0.170691,0.0
1,1,2.728791,3.256513,5.000000,4.706078,4.498998,4.442218,2.428190,1.913827,0.988645,1.336005,1.125585,0.000000,0.0
2,2,3.361582,5.000000,4.723791,4.529190,4.475832,2.583176,2.099811,1.230384,1.556811,1.359071,0.301318,0.000000,1.0
3,3,5.000000,4.723791,4.529190,4.475832,2.583176,2.099811,1.230384,1.556811,1.359071,0.301318,0.000000,0.643440,1.0
4,4,5.000000,4.794020,4.737542,2.734219,2.222591,1.302326,1.647840,1.438539,0.318937,0.000000,0.681063,1.787376,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11434576,11435404,0.833325,0.000000,0.000000,2.500000,4.722216,4.444433,3.611108,4.166649,3.333325,5.000000,2.777757,1.666649,0.0
11434577,11435405,1.086958,1.086958,3.043479,4.782604,4.565208,3.913042,4.347813,3.695646,5.000000,3.260854,2.391292,0.000000,1.0
11434578,11435406,1.399994,3.199997,4.799995,4.599991,3.999996,4.399986,3.799992,5.000000,3.399982,2.599983,0.399990,0.000000,0.0
11434579,11435407,3.499998,4.833330,4.666659,4.166664,4.499989,3.999994,5.000000,3.666653,2.999987,1.166661,0.833336,0.000000,0.0


In [ ]:
data = data.drop(["Unnamed: 0"],axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

(10291122, 12) (10291122,)
(1143459, 12) (1143459,)


,12_candels_close,11_candels_close,10_candels_close,9_candels_close,8_candels_close,7_candels_close,6_candels_close,5_candels_close,4_candels_close,3_candels_close,2_candels_close,1_candels_close,suggestion
0,0.949209,2.169026,2.826810,5.000000,4.633638,4.375520,4.304747,1.794337,1.153204,0.000000,0.432971,0.170691,0.0
1,2.728791,3.256513,5.000000,4.706078,4.498998,4.442218,2.428190,1.913827,0.988645,1.336005,1.125585,0.000000,0.0
2,3.361582,5.000000,4.723791,4.529190,4.475832,2.583176,2.099811,1.230384,1.556811,1.359071,0.301318,0.000000,1.0
3,5.000000,4.723791,4.529190,4.475832,2.583176,2.099811,1.230384,1.556811,1.359071,0.301318,0.000000,0.643440,1.0
4,5.000000,4.794020,4.737542,2.734219,2.222591,1.302326,1.647840,1.438539,0.318937,0.000000,0.681063,1.787376,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11434576,0.833325,0.000000,0.000000,2.500000,4.722216,4.444433,3.611108,4.166649,3.333325,5.000000,2.777757,1.666649,0.0
11434577,1.086958,1.086958,3.043479,4.782604,4.565208,3.913042,4.347813,3.695646,5.000000,3.260854,2.391292,0.000000,1.0
11434578,1.399994,3.199997,4.799995,4.599991,3.999996,4.399986,3.799992,5.000000,3.399982,2.599983,0.399990,0.000000,0.0
11434579,3.499998,4.833330,4.666659,4.166664,4.499989,3.999994,5.000000,3.666653,2.999987,1.166661,0.833336,0.000000,0.0
